In [3]:
import keras
import tensorflow as tf
import numpy as np

Using TensorFlow backend.


In [4]:
def l2_norm(x, axis=1):
    """l2 norm"""
    norm = tf.norm(x, axis=axis, keepdims=True)
    output = x / norm
    return output

class Extractor(object):
    def __init__(self, model):
        self.model = model

    def predict(self, imgs):
        imgs = imgs / 255.0
        embeds = l2_norm(self.model(imgs))
        return embeds

    def __call__(self, x):
        return self.predict(x)

In [5]:
model = keras.models.load_model("extractor_2.h5")
# model.compile()
# e = Extractor(model)

In [6]:
# https://www.tensorflow.org/lite/performance/post_training_quantization
# Fake a representative dataset because I'm l-a-z-y
def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, 244, 244, 3)
      yield [data.astype(np.float32)]

In [7]:
import tensorflow as tf

## Dynamic range quantization

In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

INFO:tensorflow:Assets written to: /var/folders/l0/h__2c37508b8pl19zp232ycr0000gn/T/tmpbude57rq/assets


In [9]:
with open('extractor_2_dynrange.tflite', 'wb') as f:
    f.write(tflite_quant_model)

## Full integer quantization

In [16]:
# def representative_dataset():
#     for data in tf.data.Dataset.from_tensor_slices((images)).batch(1).take(100):
#         yield [tf.dtypes.cast(data, tf.float32)]

def representative_dataset():
    for _ in range(100):
        data = np.random.rand(1, 244, 244, 3)
        yield [data.astype(np.float32)]

In [17]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
tflite_quant_model = converter.convert()

INFO:tensorflow:Assets written to: /var/folders/l0/h__2c37508b8pl19zp232ycr0000gn/T/tmpbw2em_fm/assets


INFO:tensorflow:Assets written to: /var/folders/l0/h__2c37508b8pl19zp232ycr0000gn/T/tmpbw2em_fm/assets


RuntimeError: Attempting to resize dimension 1 of tensor 0 with value 112 to 244. ResizeInputTensorStrict only allows mutating unknown dimensions identified by -1.

In [ ]:
with open('extractor_2_fullint.tflite', 'wb') as f:
    f.write(tflite_quant_model)

## Float16 quantization

In [10]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()

INFO:tensorflow:Assets written to: /var/folders/l0/h__2c37508b8pl19zp232ycr0000gn/T/tmpondmqxlm/assets


INFO:tensorflow:Assets written to: /var/folders/l0/h__2c37508b8pl19zp232ycr0000gn/T/tmpondmqxlm/assets


In [11]:
with open('extractor_2_float16.tflite', 'wb') as f:
    f.write(tflite_quant_model)

# Opening a TFLite model

In [12]:
model = keras.models.load_model("extractor_2.h5")
model.compile()

In [13]:
random_image = np.random.rand(1, 112, 112, 3)
features = model.predict(random_image)
features[0][:10]

array([-0.5515816 ,  0.19813654, -0.5685243 , -0.92722976,  0.16133478,
        0.0358324 ,  1.3602583 ,  0.705272  , -0.04875009,  1.256073  ],
      dtype=float32)

In [14]:
interpreter = tf.lite.Interpreter("extractor_2_float16.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [62]:
isinstance(interpreter, tf.lite.Interpreter)

True

In [55]:
input_data = np.array(random_image, dtype=np.float32)

In [60]:
interpreter.set_tensor(input_details[0]['index'], input_data)

ValueError: Cannot set tensor: Got value of type NOTYPE but expected type FLOAT32 for input 0, name: input_image 

In [57]:
interpreter.invoke()

In [59]:
output_data = interpreter.get_tensor(output_details[0]['index'])
features[0][:10]

array([-1.1575693 ,  1.408076  , -1.5680199 , -1.9638401 ,  6.3163443 ,
       -0.8732255 ,  1.181337  , -0.02215778, -1.8622189 , -0.6213044 ],
      dtype=float32)

[{'name': 'input_image',
  'index': 0,
  'shape': array([  1, 112, 112,   3], dtype=int32),
  'shape_signature': array([ -1, 112, 112,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]